# 2.1. Recursion and List Data-Types

In [1]:
// TypeScript Jupyter extension
import * as tslab from "tslab";

// CSC 600 Libraries
import { drawList, drawCallStack, requireCytoscape, requireCarbon} from "./lib/draw";
import * as introspect from "./lib/introspect";
import * as list from "./lib/list";

requireCarbon();
requireCytoscape();

## Where Were We?

Concept Roadmap:

1. **Bottom-up, i.e., building blocks of languages.** (TODAY and next 3 weeks)
    - **Data-Types + Recursion** (this week)
    - First-Class Functions or References + State
2. Top-down, i.e., using building blocks.
3. *Meta-theory.* (LAST TIME, introduced TypeScript and BNF)

## Goal

1. Introduce the concept of **(algebraic) data-types** (**ADT**) using **lists**.
2. Examine how **recursive functions** operate on lists.
    
Recall language template

1. Input/Output:
2. **Data: list data-type**
3. **Code: recursive function**

## Outline

- Iteration vs. Recursion via example
- Lists
    * List as data-type
    * Recursive functions on lists

## Iteration vs. Recursion

Let's just look at a problem ...

### Coding Problem

Given an array of numbers, find the smallest element in the array.

In [2]:
// Example 1
const xs = [10000000000];
console.log("Smallest", 10000000000);

Smallest 10000000000


In [3]:
// Example 2
const xs = [2, 1];
console.log("Smallest", 1);

Smallest 1


In [4]:
// Example 3
const xs = [];
console.log("Smallest", undefined);

Smallest undefined


In [5]:
// Example 4
const xs = [32, 21, 13, 90];
console.log("Smallest", 13);

Smallest 13


### An iterative solution

I know most of you can code this in your sleep ...

In [6]:
function iterFindSmallest(xs: number[]): number | undefined {
    if (xs.length === 0) { // Question: == vs. ===?
        return undefined;
    }
    
    let smallest = xs[0]; // State
    for (let i=1; i < xs.length; i++) {   // Loops
        smallest = Math.min(smallest, xs[i]);
    }
    return smallest;
}

iterFindSmallest([32, 21, 13, 90]);

13


```












For the moment, let's pretend we did
not have loops and state. What would
you do? Please post in chat.













```

### A general problem solving "strategy" in computer science

- I can't solve the problem you gave me.
- However I can solve a simpler version of the problem you gave me.
- Any maybe the solution to the simpler problem will give me a hint for how to solve the "harder" problem.

### A solution for up to length 1 arrays

In [7]:
function findSmallestUpTo1(xs: number[]): number | undefined {
    if (xs.length === 0) { // Deal with empty array
        return undefined;
    } 
    
    // Body
    const smallest1 = xs[0];
    if (xs.length === 1) {
        return smallest1;
    } else {
        throw Error("Sad.");
    }
}

In [8]:
console.log("Smallest", findSmallestUpTo1([]));
console.log("Smallest", findSmallestUpTo1([32]));
try {
    console.log("Smallest", findSmallestUpTo1([32, 21]));
} catch (err) {
    tslab.display.html(`<p style="color:red;"> ${err}</p>`);
}

Smallest undefined
Smallest 32


Error: Sad.

### A solution for up to length 2 arrays

In [9]:
function findSmallestUpTo2(xs: number[]): number | undefined {
    if (xs.length === 0) { // Deal with empty array
        return undefined;
    } 
    
    const smallest1 = xs[0];
    if (xs.length === 1) {
        return smallest1;
    } else {
        // This used to be "throw Error("Not my problem ...");"
        // Now we essentially copy-and-pasted the body of findSmallestUpTo1 here and
        // 1. changed same variable names
        // 2. used Math.min to the work of computing the minimum
        const smallest2 = Math.min(smallest1, xs[1]);
        if (xs.length === 2) {
            return smallest2;
        } else { 
            throw Error("Sad.");
        }
    }
}

In [10]:
console.log("Smallest", findSmallestUpTo2([]));
console.log("Smallest", findSmallestUpTo2([32]));
console.log("Smallest", findSmallestUpTo2([32, 21]));
try {
    console.log("Smallest", findSmallestUpTo2([32, 21, 13]));
} catch (err) {
    tslab.display.html(`<p style="color:red;"> ${err}</p>`);
}

Smallest undefined
Smallest 32
Smallest 21


Error: Sad.

### A solution for up to length 3 arrays ...

In [11]:
function findSmallestUpTo3(xs: number[]): number | undefined {
    if (xs.length === 0) { // Deal with empty array
        return undefined;
    } 
    
    const smallest1 = xs[0];
    if (xs.length === 1) {
        return smallest1;
    } else {
        const smallest2 = Math.min(smallest1, xs[1]);
        if (xs.length === 2) {
            return smallest2;
        } else { 
            const smallest2 = Math.min(smallest1, xs[1]);
            if (xs.length === 3) {
                return smallest2;
            } else { 
                // This used to be "throw Error("Sad.");"
                // Now we essentially copy-and-pasted the body of findSmallestUpTo1 here and
                // 1. changed same variable names
                // 2. used Math.min to the work of computing the minimum
                const smallest3 = Math.min(smallest2, xs[2]);
                if (xs.length === 3) {
                    return smallest3;
                }
                else {
                    throw Error("Sad.");   
                }
            }
        }
    }
}

In [12]:
console.log("Smallest", findSmallestUpTo3([]));
console.log("Smallest", findSmallestUpTo3([32]));
console.log("Smallest", findSmallestUpTo3([32, 21]));
console.log("Smallest", findSmallestUpTo3([32, 21, 13]));
try {
    console.log("Smallest", findSmallestUpTo3([32, 21, 13, 90]));
} catch (err) {
    tslab.display.html(`<p style="color:red;"> ${err}</p>`);
}

Smallest undefined
Smallest 32
Smallest 21
Smallest 21


Error: Sad.

### Summary of solution to simpler problem

- We did not solve the problem for arbitrary length arrays.
- But we saw that if we had an unfolding mechanism that does "copy-paste" we could solve the problem.
- A recursive function is a language feature that gives you this unfolding mechanism.
    * **Main idea**: a recursive solution gives a solution to a problem by combining solutions to smaller instances of the **same** problem.

In [13]:
function findSmallest(xs: number[]): number | undefined {
    if (xs.length === 0) {
        return undefined;
    }
    
    if (xs.length === 1) { // Base case
        return xs[0];
    } else {               // Recursive (or inductive) case
        // xs.slice(1) is the smaller instance
        // Math.min combines the solution to the smaller instance
        // Intuition: I copy-paste myself whereever I reference myself
        return Math.min(xs[0], findSmallest(xs.slice(1)));
    }
}

findSmallest([32, 21, 13, 90])

13


### Hold On

This is an artificially constructed problem ... I do have loops and state. Do I really need to learn recursion?

- Next time we'll look at what I hope to be a more compelling case for recursion.

For now, I'd like to introduce the idea of a an algebraic data-type.

- Earlier when we introduced the template for "learning a programming language", we had the idea that the *data* that your language defines informs the kinds of *code* / language constructs that your language should provide.
- **(algebraic) data-types** are the corresponding language construct for recursive functions.

## Lists

- We'll use the list data-type to give the "simplest" example of a data-type.
    * A data-type describes different ways of building a data-structure.
    * Each way of building up the data-structure has an associated *constructor*
- As a programmer, you will need to understand (1) how to read existing data-types and (2) implement your own data-types.

In [14]:
const arr = [3, 2, 1];
console.log(arr);

// The corresponding array as a list [3, 2, 1]
drawList(list.ls3);

[ 3, 2, 1 ]


In [15]:
// The list data-type
enum _List { NIL, CONS };
type List<T> =
  {tag: _List.NIL}                                // An empty list
| {tag: _List.CONS, contents: T, rest: List<T>};  // A contents cell followed by a list

In [16]:
function Nil<T>(): List<T> {
    // Helper function for constructing an empty list
    return {tag: _List.NIL};
}

function Cons<T>(x: T, ls: List<T>): List<T> {
    // Helper function for adding an element to the front of a list
    return {tag: _List.CONS, contents: x, rest: ls};
}

In [17]:
const ls0 = Nil();
const ls1 = Cons(1, Nil());
const ls2 = Cons(2, ls1);
const ls3 = Cons(3, ls2);
const ls4 = Cons(4, ls3);
const ls5 = Cons(5, ls4);

In [18]:
drawList(ls0);

In [19]:
ls0

{ tag: 0 }


In [20]:
drawList(ls1)

In [21]:
ls1 // Notice how the entirety of ls0 is "nested" inside ls1

{ tag: 1, contents: 1, rest: { tag: 0 } }


In [22]:
drawList(ls2)

In [23]:
ls2 // Notice how the entirety of ls1 is "nested" inside ls2

{
  tag: 1,
  contents: 2,
  rest: { tag: 1, contents: 1, rest: { tag: 0 } }
}


In [24]:
drawList(ls3)

In [25]:
ls3

{
  tag: 1,
  contents: 3,
  rest: {
    tag: 1,
    contents: 2,
    rest: { tag: 1, contents: 1, rest: [Object] }
  }
}


### Simple Functions on Lists

- Some examples of simple (i.e., not-recursive) functions on lists

In [26]:
function head<T>(ls: List<T>): T {
    // A desctructor for a list
    switch (ls.tag) {
        case _List.NIL: {
            throw Error("Empty list ...");
        }
        case _List.CONS: {
            return ls.contents;
        }
    }
}

In [27]:
drawList(ls3)

In [28]:
head(ls3)

3


In [29]:
function tail<T>(ls: List<T>): List<T> {
    // A list destructor
    switch (ls.tag) {
        case _List.NIL: {
            throw Error("Empty list ...");
        }
        case _List.CONS: {
            return ls.rest;
        }
    }
}

In [30]:
drawList(tail(ls3))

In [31]:
tail(ls2)

{ tag: 1, contents: 1, rest: { tag: 0 } }


### Recursive Functions on Lists

- We just saw simple operations on lists: building them and taking them apart.
- What about recursion? 

#### Problem

- Suppose you want to compute the length of a list.

#### First an iterative solution

Recall with arrays and dictionaries, i.e., collections of items, we used looping mechanisms to examine the contents of collections.
```
const xs = [1, 2, 3];
for (let x of xs) {
  console.log(x)
}
```

In [32]:
function iterLength<T>(ls: List<T>): number {
    let lsHead = ls;
    let ans = 0;  // State
    let iter = 0;  
    while (true) {
        // Visualize
        console.log(`iterLength iteration: ${iter}      current answer: ${ans}`);
        drawList(lsHead);
        switch (lsHead.tag) {
            case _List.NIL: {
                return ans;
            }
            case _List.CONS: {
                ans += 1;
                iter += 1;
                lsHead = lsHead.rest;
            }
        }
    }
}

In [33]:
iterLength(ls3)

iterLength iteration: 0      current answer: 0


iterLength iteration: 1      current answer: 1


iterLength iteration: 2      current answer: 2


iterLength iteration: 3      current answer: 3


3


#### Pretending again that we don't have loops, solving a simpler problem

Let's try solving the simpler problem for up to length 2 lists.

In [34]:
function lengthUpTo1<T>(ls: List<T>): number {
    const head = ls;   // List we are looking at
    const ans = 0;     // Contains the answer for length 0 lists
    switch (head.tag) {
        case _List.NIL: {
            return ans;
        }
        case _List.CONS: {
            const head1 = head.rest;  // List we are looking at
            const ans1 = 1 + ans;     // Contains the answer for length 1 lists
            switch (head1.tag) {
                case _List.NIL: {
                    return ans1;
                }
                case _List.CONS: {
                    throw Error("Sad.")
                }
            }
        }
    }
}

In [35]:
console.log("Length", lengthUpTo1(ls0));
console.log("Length", lengthUpTo1(ls1));
try {
    console.log(lengthUpTo1(ls2));
} catch(err) {
    tslab.display.html(`<p style="color:red;"> ${err}</p>`);
}

Length 0
Length 1


Error: Sad.

In [36]:
function lengthUpTo2<T>(ls: List<T>): number {
    const head = ls;
    const ans = 0;
    switch (head.tag) {
        case _List.NIL: {
            return ans;
        }
        case _List.CONS: {
            const head1 = head.rest;
            const ans1 = 1 + ans;
            switch (head1.tag) {
                case _List.NIL: {
                    return ans1;
                }
                case _List.CONS: {
                    // This used to be throw Error("Sad.");
                    // We copy and pasted the entire body of lengthUpTo1 in here and changed some variables around.
                    const head2 = head1.rest;
                    const ans2 = ans1 + 1;
                    switch (head2.tag) {
                        case _List.NIL: {
                            return ans2;
                        }   
                        case _List.CONS: {
                            // Keep unfolding ...
                            throw Error("Sad.");
                        }
                    }
                }
            }
        }
    }
}
    
lengthUpTo2(ls2)

2


In [37]:
console.log("Length", lengthUpTo2(ls0));
console.log("Length", lengthUpTo2(ls1));
console.log("Length", lengthUpTo2(ls2));
try {
    console.log("Length", lengthUpTo2(ls3));
} catch(err) {
    tslab.display.html(`<p style="color:red;"> ${err}</p>`);
}

Length 0
Length 1
Length 2


Error: Sad.

#### A recursive solution

In [38]:
function length<T>(ls: List<T>): number {
    const head = ls;
    const ans = 0;
    switch (ls.tag) {
        case _List.NIL: {
            // Base case: no call to itself.
            return ans;
        }
        case _List.CONS: {
            // Recursive case: call to itself. "Copy-paste the definition of length"
            const ansRest = length(ls.rest);
            const ansRestplus1 = 1 + ansRest;
            return ansRestplus1;
        }
    }
}

In [39]:
console.log("Length", length(ls0));
console.log("Length", length(ls1));
console.log("Length", length(ls2));
try {
    console.log("Length", length(ls5));
} catch(err) {
    tslab.display.html(`<p style="color:red;"> ${err}</p>`);
}

Length 0
Length 1
Length 2
Length 5


#### How does the recursive solution work?

Let's do a **call stack** trace.

In [40]:
drawList(ls1)

In [41]:
const res = introspect.traceCallStack(length, exports);
console.log(res.func(ls0));
drawCallStack(res.stack);

0


In [42]:
const res = introspect.traceCallStack(length, exports);
console.log(res.func(ls1));
// Notice the nesting structure: we took the previous diagram and wrapped it on both ends,
// on one end with CALL and on the other end with RET.
drawCallStack(res.stack);

1


In [43]:
const res = introspect.traceCallStack(length, exports);
console.log(res.func(ls2));
// Notice the nesting structure: we took the previous diagram and wrapped it on both ends,
// on one end with CALL and on the other end with RET.
drawCallStack(res.stack);

2


In [44]:
function recLengthUpTo2<T>(ls: List<T>): number { // 0th Unfolding
    let head0 = ls; // CALL(ls2)
    let tmp0;       // put answer of CALL(ls2) here
    switch (head0.tag) {
        case _List.NIL: {
            tmp0 = 0;
            break;
        }
        case _List.CONS: { // 1st Unfolding
            let head1: List<T> = head0.rest; // CALL(ls1)
            let tmp1;                        // put answer of CALL(ls1) here
            switch (head1.tag) {
                case _List.NIL: {
                    tmp1 = 0;
                    break;
                }
                case _List.CONS: { // 2nd Unfolding
                    let head2: List<T> = head1.rest // CALL(ls0)
                    let tmp2;                       // put answer of CALL(ls0) here
                    switch (head2.tag) {
                        case _List.NIL: {
                            tmp2 = 0; // RET(0)
                            break;
                        }
                        case _List.CONS: {
                            throw Error("Not impelmented ...");
                        }
                    }
                    // Assuming I know the answer of CALL(ls0), compute the answer to CALL(ls1)
                    tmp1 = 1 + tmp2; // RET(1)
                    break;
                }
            }        
            // Assuming I know the answer of CALL(ls1), compute the answer to CALL(ls2)
            tmp0 = 1 + tmp1; // RET(2)
            break;
        }
    }
    return tmp0;
}

recLengthUpTo2(ls2)

2


### Iteration vs. Recursion


In [45]:
const res = introspect.traceCallStack(iterLength, exports);
console.log(res.func(ls2));
drawCallStack(res.stack);

iterLength iteration: 0      current answer: 0


iterLength iteration: 1      current answer: 1


iterLength iteration: 2      current answer: 2


2


What are the similarities/differences between the iterative approach and the recursive approach for length?

- Iteration unfolded from the front of the list.
    * Build up answer to the length question by traversing the list.
    * Call stack trace has 1 CALL-RET pair.
- Recursion unfolded from the back of the list.
    * Build up answer to the length question by using answers to the length question on smaller lists.
    * Call stack trace has N, where N is the length of the list, CALL-RET pairs.

### Can we rewrite the recursive solution?

In [46]:
// acc stands for accumulator
function accLengthHelper<T>(acc: number, ls: List<T>): number {
    switch (ls.tag) {
        case _List.NIL: {
            // End of iteration
            return acc;
        }
        case _List.CONS: {
            return accLengthHelper(acc + 1, ls.rest);
        }
    }
}

In [47]:
const res = introspect.traceCallStack(accLengthHelper, exports);
console.log(res.func(0, ls3));
drawCallStack(res.stack); // Notice how the returns do not need the previous solution anymore

3


### Tail-Call Optimization

- Because none of the later return statements need the computed values, they can be optimized away. This is called tail-call optimization.
- TypeScript does not have tail-call optimization.
- Takeaway: you should be mindful when writing recursive functions because it can use many more stack frames than an iterative function.

### Are there benefits to using recursion?

- Suppose you want to print out a more compressed but still readable form of a list.

In [48]:
// A side-note on string-interpoloation
console.log(`${1} ${2}`);
console.log(1 + " " + 2);

1 2
1 2


In [49]:
ls2

{
  tag: 1,
  contents: 2,
  rest: { tag: 1, contents: 1, rest: { tag: 0 } }
}


In [50]:
function listToString<T>(ls: List<T>): string {
    switch (ls.tag) {
        case _List.NIL: {
            // What is the "answer" for an empty list?
            return "()";
        }
        case _List.CONS: {
            // Assumtion: listToString(ls.rest) "magically" gives me the correct answer on the rest of the list
            const ansOnRestOfList = listToString(ls.rest);
            // How do I construct the correct answer for the larger list?
            // "(ls.contents ansOnRestOfList)"
            return `(${ls.contents} ${ansOnRestOfList})`;
        }
    }
}

listToString(ls2)

(2 (1 ()))


In [51]:
const res = introspect.traceCallStack(listToString, exports);
res.func(ls0)
drawCallStack(res.stack)

In [52]:
const res = introspect.traceCallStack(listToString, exports);
res.func(ls1)
drawCallStack(res.stack)

In [53]:
const res = introspect.traceCallStack(listToString, exports);
res.func(ls2)
drawCallStack(res.stack)

### Challenge: Try writing this iteratively!

How would you implement `listToString` iteratively? (Hint: recrusion unfolds the list starting from the back.)

In [54]:
function iterListToString<T>(ls: List<T>): string {
    let lsHead = ls;
    let callStack: string[] = [];
    let flag = true;
    while (flag) {
        switch (lsHead.tag) {
            case _List.NIL: {
                flag = false;
                break;
            }
            case _List.CONS: {
                callStack.push(lsHead.contents.toString());  // Calling function means push argument onto stack
                console.log("Call Stack", callStack);
                lsHead = lsHead.rest;
                break;
            }
        }
    }
    
    callStack.push(`()`);
    while (callStack.length > 1) {
        const rest = callStack.pop();                        // Pop to get answer on smaller list from stack
        const contents = callStack.pop();                    // Pop to get argument for current list
        callStack.push(`(${contents} ${rest})`);             // Push answer on current list back onto stack
    }
    return callStack.pop();                                  // Pop to get answer on original list
}

iterListToString(ls2);

Call Stack [ '2' ]
Call Stack [ '2', '1' ]
(2 (1 ()))


## Back to the Motivating Problem

In [55]:
function findSmallest(xs: number[]): number | undefined {
    if (xs.length === 0) {
        return undefined;
    } else if (xs.length === 1) {
        return xs[0];
    } else {
        // Assumption: findSmallest(xs.slice(1)) "magically" gives me the answer on the rest of the array.
        const ansOnRestOfArray = findSmallest(xs.slice(1));
        // How do I come up with the answer given that I know the smallest number in the rest of the array?
        return Math.min(xs[0], ansOnRestOfArray);
    }
}

findSmallest(xs)

13


In [56]:
const res = introspect.traceCallStack(findSmallest, exports);
res.func(xs)
drawCallStack(res.stack)

In [60]:
function recFindSmallestUpTo5(xs: number[]): number | undefined {
    // 0th unfolding
    if (xs.length === 0) {
        return undefined;
    } else if (xs.length == 1) {
        return xs[0];
    } else {  // 1st unfolding
        let ans2;
        const xs2 = xs.slice(1);
        if (xs2.length === 0) {
            ans2 = undefined;
        } else if (xs2.length === 1) {
            ans2 = xs2[0];
        } else {  // 2nd unfolding
            let ans3;
            const xs3 = xs2.slice(1);
            if (xs3.length === 0) {
                ans3 = undefined;
            } else if (xs3.length === 1) {
                ans3 = xs3[0];
            } else {  // 3rd unfolding
                let ans4;
                const xs4 = xs3.slice(1);
                if (xs3.length === 0) {
                    ans4 = undefined;
                } else if (xs4.length === 1) {
                    ans4 = xs4[0];
                } else {  // 4th unfolding
                    let ans5;
                    const xs5 = xs4.slice(1);
                    if (xs5.length === 0) {
                        ans5 = undefined;
                    } else if (xs5.length === 1) {
                        ans5 = xs5[0];
                    } else {  // 5th unfolding
                        throw Error("Sad.");
                    }
                    ans4 = Math.min(xs4[0], ans5);
                }
                ans3 = Math.min(xs3[0], ans4);
            }    
            ans2 = Math.min(xs2[0], ans3);
        }
        return Math.min(xs[0], ans2);
    }
}

recFindSmallestUpTo5(xs)

13


## Story for Today?


1. We started with the problem: find the smallest element in an array.
2. We used the familiar iterative solution.
3. We constrained ourselves to solve it without iteration.
4. We further constrained ourselves by solving the problem for fixed size arrays.
5. This led us to discover **recursion**
    * A way to succinctly express unfolding code an arbitrary number of times.
    * Recursion has "nested" structure. We wrap the "unfolding" with the code in the current function body and up to the return statement.
6. We found that the "data" corresponding to recursive-style "code" was an **(algebraic) data-type**.
7. We then compared recursion with iteration and discovered a call stack.